In [2]:
import pandas as pd
import emoji
import ast
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tibi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
## Incarcare dataset
df = pd.read_csv('dataset/train_data.csv')
df.head()

In [17]:
def text_without_emojis():
    text_noemoji = []
    for i in df.index:
        line = df['Text'].iloc[i]
        emojis = emoji.emoji_list(line)
        for emoji_dict in emojis:
            emoji_str = emoji_dict.get('emoji')
            line = re.sub(emoji_str, '', line)
        text_noemoji.append(line)
    return text_noemoji

In [18]:
def text_without_mentions():
    text_nomentions = [re.sub(r'@\S+', '', line) for line in df['Text']]
    return text_nomentions

In [19]:
def weird_characters():
    chars = {character for line in df['Text'] for character in line if not re.search(r'[a-zA-Z0-9șțîâăȘȚÂĂÎ\s_-]+', character)}
    return chars

In [20]:
def check_with_dictionary(file_name):
    text_cleaned = []
    with open(file_name, "r", encoding='UTF-8') as data:
        dictionary = ast.literal_eval(data.read())
        # print(dictionary)
        # print(type(dictionary))
        for line in df['Text']:
            for key in dictionary:
                if key != dictionary[key]:
                    while line.find(key) != -1:
                        line = line.replace(key, dictionary[key])
            text_cleaned.append(line)
    return text_cleaned

In [21]:
def clean_non_ascii():
    text_cleaned = []
    for line in df['Text']:
        tokens = word_tokenize(line)
        clean_tokens = [token for token in tokens if token.isascii() or re.search(r'[/SșțîâăȘȚÂĂÎ/S]',token)]
        clean_text = " ".join(clean_tokens)
        text_cleaned.append(clean_text)
    return text_cleaned

In [22]:
def clean_non_alphanumerical():
    text_cleaned = []
    dictionary = weird_characters()
    for line in df['Text']:
        for elem in dictionary:
            while line.find(elem) != -1:
                line = line.replace(elem, '')
        text_cleaned.append(line)
    return text_cleaned

In [23]:
def lowercase_text():
    text_lowercase = []
    for line in df['Text']:
        line = line.replace('_', ' ')
        if not re.search(r'[ȘȚÂĂÎ]+', line):
            line = line.lower()
        else:
            line = line.replace("Ț", "ț").replace("Ș", "ș").replace("Î", "î").replace("Ă", "ă").replace("Â", "â")
        text_lowercase.append(line)

    return text_lowercase

In [ ]:
df['Text'] = text_without_emojis()
df['Text'] = text_without_mentions()
# df['Text'] = check_with_dictionary('character map/full_replace.txt')
# df['Text'] = clean_non_ascii()
# df['Text'] = clean_non_alphanumerical()
# df['Text'] = lowercase_text()

df.to_csv('dataset/train_data_noemoji_nomentions.csv', index=False)

In [ ]:
## Testare dataset
#df = pd.read_csv('dataset/train_data_noemoji.csv')
#df = pd.read_csv('dataset/train_data_noemoji_nomentions.csv')
df.head(10)